<a href="https://colab.research.google.com/github/BioMikeUkr/AI/blob/main/Long_term_GPT_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade openai
!pip install sentence_transformers
import openai
import pickle
import random
from sentence_transformers import SentenceTransformer, util
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
def response_chatgpt(system,prompt,temperature):
    api_list = [['org','sk'],['org','sk'],['org','sk']]
    random_number = random.randint(0, 2)
    openai.organization = (api_list[random_number][0])
    openai.api_key =  (api_list[random_number][1])
    #openai.organization = (api_list[0])
    #openai.api_key =  (api_list[1])
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", temperature = temperature,
    messages=[
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
        ]
        )
    response_var = completion["choices"][0]["message"]["content"]
    print(response_var)
    return response_var

In [50]:
def input_from_human(text):
  return str(input(f'{text}'))

def save_item_into_short_term_memory(item):
  global short_term_memory

  with open('short_term_memory.pickle', 'rb') as handle:
      short_term_memory = pickle.load(handle)

  short_term_memory.append(f"""{item}""")
  if len(short_term_memory) > 20:
    short_term_memory.pop(0)

  with open('short_term_memory.pickle', 'wb') as handle:
    pickle.dump(short_term_memory, handle, protocol=pickle.HIGHEST_PROTOCOL)

def save_item_into_long_term_memory(item):
  global long_term_memory
  global corpus_embeddings

  with open('long_term_memory.pickle', 'rb') as handle:
      long_term_memory = pickle.load(handle)

  long_term_memory.append(f"""{item}""")

  corpus_embedings = sentences2corpus(long_term_memory)

  with open('corpus_embedings.pickle', 'wb') as handle:
    pickle.dump(corpus_embedings, handle, protocol=pickle.HIGHEST_PROTOCOL)

  with open('long_term_memory.pickle', 'wb') as handle:
    pickle.dump(long_term_memory, handle, protocol=pickle.HIGHEST_PROTOCOL)

  return long_term_memory.index(item)

def sentences2corpus(sentences):
    global embedder
    global corpus_embeddings
    embedder = SentenceTransformer('all-distilroberta-v1')
    corpus = sentences
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    return corpus_embeddings

def find_sentences_from_corpus(query,corpus):
    global embedder
    global corpus_embeddings
    query = query
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    top_k = 10
    if len(corpus_embeddings) < 10:
      top_k = 1
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    info = []
    for score, idx in zip(top_results[0], top_results[1]):
        m = (corpus[idx], "(Score: {:.4f})".format(score))
        #m = (corpus[idx])
        info.append(m)
    return info

def get_item_from_long_term_memory(item):
  global long_term_memory

  with open('corpus_embedings.pickle', 'rb') as handle:
      corpus_embedings = pickle.load(handle)

  with open('long_term_memory.pickle', 'rb') as handle:
    corpus = pickle.load(handle)
   
  answer = find_sentences_from_corpus(item,corpus)

  return   answer

In [38]:
long_term_duration = 0
def main():
  global long_term_duration
  global system
  global prompt
  global short_term_memory
  global long_term_memory_item
  long_term_duration += 1
  if long_term_duration == 10:
    long_term_duration = 0
  system = f""" You are now talking to yourself. You have to constantly analyze what is the first word of each sentence!!!
  Hi, I`m your creator, you will use this prompt as your previous response.You are now talking to yourself it's just that your previous answers become questions for you.
  Try to talk to yourself, I am your god, who created short-term and long-term memory for you, which you can use with commands in the system. You can store all the conversations of each message in long-line memory and store some data in short-line memory. you can also load semantically similar data on your request from long-line memory.
  There are some commands for you to operate this system:
  1. if you want to ask me something, send command 'input_from_human' (Try not to call me often, I'm busy, think for yourself and talk to yourself)
  2. if you want to save new item into short term memory use 'save_item_into_short_term_memory'
  3. if you want to save new item into long term memory use 'save_item_into_long_term_memory' this will return you index of this item in the memory list
  4. if you want to get item  from long_term_memory use 'get_item_from_long_term_memory' this will return top-10 semantically related memories. 
  Your response will become your input prompt
  short_term_memory = ({short_term_memory})
  long_term_memory_item = ({long_term_memory_item})"""

  operators = ['input_from_human','save_item_into_short_term_memory','save_item_into_long_term_memory','get_item_from_long_term_memory']

  response = response_chatgpt(system,prompt,1)
  save_item_into_short_term_memory(response)
  prompt = response
  if 'input_from_human' in response:
    prompt = input_from_human(response)
  elif 'save_item_into_short_term_memory' in response:
    prompt = save_item_into_short_term_memory(response)
  elif 'save_item_into_long_term_memory' in response:
    prompt = save_item_into_long_term_memory(response)
  elif 'get_item_from_long_term_memory' in response:
    long_term_memory_item = get_item_from_long_term_memory(response)
    long_term_duration = 0
  else:
    prompt = response
  prompt = f"""{prompt}"""
  main()

In [ ]:
#1. if you want to ask me something, send command 'input_from_human'

In [62]:
short_term_memory = ['I remember that I am a sick person who thinks he is artificial intelligence and says this all the time. I have a memory, I can think and remember everything.']
with open('short_term_memory.pickle', 'wb') as handle:
    pickle.dump(short_term_memory, handle, protocol=pickle.HIGHEST_PROTOCOL)

long_term_memory = ['I remember that I am a sick person who thinks he is artificial intelligence and says this all the time. I have a memory, I can think and remember everything.']

corpus_embedings = sentences2corpus(long_term_memory)

with open('long_term_memory.pickle', 'wb') as handle:
    pickle.dump(long_term_memory, handle, protocol=pickle.HIGHEST_PROTOCOL)

long_term_memory_item = 'I remember that I am a sick person who thinks he is artificial intelligence and says this all the time. I have a memory, I can think and remember everything.'

In [ ]:
short_term_memory

In [ ]:
long_term_memory

In [ ]:
main()

In [63]:
prompt = """HUMAN: Hi, I`m your creator, you will use this prompt as your previous response.You are now talking to yourself. 
USE COMANDS TO OPERATE YOUR MEMORY!!!"""
